In [ ]:
cd ../

In [ ]:
%matplotlib inline
import numpy as np
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import sys
sys.path.append('src/')
from tools.io import load_tvsum_mat

In [ ]:
def accum_eval(pred, gt):
    total = gt.mean(axis=0).sum()
    x = np.argsort(pred)[::-1]
    
    y = [0]
    for i in range(x.size):
        cur_score = y[-1]
        y.append(cur_score + gt[:, x[i]].mean())
    y = np.asarray(y[1:]) / total
    return y

def best_curve(gt):
    total = gt.mean(axis=0).sum()
    x = np.argsort(gt.mean(axis=0))[::-1]
    
    y = [0]
    for i in range(x.size):
        cur_score = y[-1]
        y.append(cur_score + gt[:, x[i]].mean())

    y = np.asarray(y[1:]) / total
    
    return y

def worst_curve(gt):
    total = gt.mean(axis=0).sum()
    x = np.argsort(gt.mean(axis=0))
    
    y = [0]
    for i in range(x.size):
        cur_score = y[-1]
        y.append(cur_score + gt[:, x[i]].mean())
    y = np.asarray(y[1:]) / total
    return y

def plot_user_scores(user_anno):
    user_anno = (user_anno - 1.) / 4.
    N = len(user_anno)

    plt.figure(figsize=(5,5))

    # upper-bound
    best_y = best_curve(user_anno)
    best_auc = auc(np.linspace(0, 1, best_y.size), best_y)

    # lower-bound
    worst_y = worst_curve(user_anno)
    worst_auc = auc(np.linspace(0, 1, worst_y.size), worst_y)

    plt.fill_between(range(len(best_y)), worst_y, best_y, color='lightblue', alpha=.5)

    mean_auc = 0
    for i in range(N):
        pred = user_anno[i]
        y = accum_eval(pred, user_anno[list(range(i))+list(range(i+1, N))])
        mean_auc += auc(np.linspace(0, 1, y.size), y)
        p0 = plt.plot(y, color='lightcoral', alpha=.5)
        
    return mean_auc, best_auc, worst_auc, p0

def plot_curve_rlvsumm():
    tvsum_data = load_tvsum_mat('./data/raw/tvsum/ydata-tvsum50.mat')
    
    human_auc_summary = []
    random_auc_summary = []
    rel_human_auc = []
    rel_random_auc = []
    
    for gt in tvsum_data:
        user_anno = gt['user_anno'].T
        n_fr = user_anno.shape[1]
        N = len(user_anno)

        human_mean_auc, best_auc, worst_auc, p0 = plot_user_scores(user_anno)
        human_auc_summary.append(human_mean_auc / N)
        
        rel_human_auc.append((human_auc_summary[-1]-worst_auc) / (best_auc-worst_auc)*100)
        
        # plot curve by random scoring
        pred = np.random.random((n_fr))
        y = accum_eval(pred, user_anno)
        p1 = plt.plot(y, color='k', linestyle='--')
        random_auc = auc(np.linspace(0, 1, y.size), y)
        random_auc_summary.append(random_auc)
        
        rel_random_auc.append((random_auc-worst_auc) / (best_auc-worst_auc)*100)
        
        plt.legend((p0[0], p1[0]), ('Humans', 'Random'))
        plt.title(gt['video'])
        plt.show()
        plt.close('all')
        
    print('random:', sum(random_auc_summary)/len(random_auc_summary), sum(rel_random_auc)/len(rel_random_auc), '\n',
         'human:', sum(human_auc_summary)/len(human_auc_summary), sum(rel_human_auc)/len(rel_human_auc))

In [ ]:
sns.set_context('notebook', font_scale=1.3)
plot_curve_rlvsumm()